In [1]:
!pip install hmmlearn
!pip install python_speech_features

     |████████████████████████████████| 129 kB 5.3 MB/s 
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=1ba80de375b12a14e11f0f9d43f91a0630cee38f743c9a0aafc739426c0ba8ee
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features


In [2]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as dsp
import numpy as np
import os
import shutil
import subprocess
import random
from hmmlearn.base import ConvergenceMonitor
from hmmlearn import hmm
from sklearn import preprocessing
from scipy.io.wavfile import read
from python_speech_features import mfcc
from python_speech_features import delta

In [3]:
path1 = "D:\\xltn\\data\\{}\\{}\\file{}.wav"
path2 = "D:\\xltn\\dataset\\{}\\file{}.wav"

In [5]:
def mfcc(wav_path, delta=2):
    y, sr = librosa.load(wav_path)
    # MEL frequency cepstrum coefficient
    mfcc_feat = librosa.feature.mfcc(y = y, sr = sr, n_mfcc = 13)
    ans = [mfcc_feat]
    # Calculate the 1st derivative
    if delta >= 1:
        mfcc_delta1 = librosa.feature.delta(mfcc_feat, order = 1, mode ='nearest')
        ans.append(mfcc_delta1)
    # Calculate the 2nd derivative
    if delta >= 2:
        mfcc_delta2 = librosa.feature.delta(mfcc_feat, order = 2, mode ='nearest')
        ans.append(mfcc_delta2)

    return np.transpose(np.concatenate(ans, axis = 0),[1,0])

In [6]:
input_folder = "D:\\xltn\\dataset\\"

In [7]:
def train_model_hmm(train_dir):
    train_files = [x for x in os.listdir(train_dir) if x.endswith('.wav')]
    X = np.array([])
    for file_name in train_files:
        file_path = os.path.join(train_dir, file_name)
        try:
            features_mfcc = mfcc(file_path)
            if len(X) == 0:
                X = features_mfcc
            else:
                try:
                    X = np.append(X, features_mfcc, axis=0)
                except:
                    pass
        except:
            print(file_path)
            pass
    model = hmm.GaussianHMM(n_components=4, covariance_type='diag', n_iter=1800)
    # fit hmm model
    np.seterr(all='ignore')
    model.fit(X)
    return model

In [ ]:
# mỗi nhãn tạo một model 
hmm_models = []
for digit in os.listdir(input_folder):
    label = digit
    path = input_folder+ digit
    print(path)
    hmm_models.append((train_model_hmm(path), label))

D:\xltn\dataset\a
D:\xltn\dataset\b
D:\xltn\dataset\ban
D:\xltn\dataset\len
D:\xltn\dataset\nhay
D:\xltn\dataset\phai
D:\xltn\dataset\trai
D:\xltn\dataset\xuong


In [ ]:
def predict_hmm(hmm_models, test_file):
    features_mfcc_test = mfcc(test_file)
    max_score = -float('inf')
    predicted_label = ""
    for item in hmm_models:
        model, label = item
        score = model.score(features_mfcc_test)
        if score > max_score:
            max_score = score
            predicted_label = label
    return predicted_label

In [ ]:
test_paths = []
for path in os.listdir(input_folder):
    label = path
    arr = os.listdir(input_folder+path)
    test_arr = np.random.choice(arr,10)
    for test_file in test_arr:
        test_paths.append({
            "test_path" : input_folder+path+"\\"+test_file,
            "label": label})

In [ ]:
predict_true = 0
for test in test_paths:
    predict_label = predict_hmm(hmm_models,test["test_path"])
    if predict_label == test["label"]:
        predict_true +=1

In [ ]:
predict_true/len(test_paths)

0.9375